In [10]:
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import interp1d
import scipy.stats as stats
from statsmodels.tsa.stattools import acf
import pandas as pd
import scipy.optimize
from statsmodels.tsa.arima_process import ArmaProcess
import pandas as pd
sns.set()

In [19]:
def yule_walker_estimation(g_0, g_1, g_2):
    A = np.array([[g_0, g_1], [g_1, g_0]])
    b = np.array([g_1, g_2])
    phi = np.linalg.solve(A, b)
    return phi[0], phi[1]

In [20]:
g_0 = 1

In [22]:
def simulate_ar2(phi1, phi2, n_simulations=1000, n=100):
    ar = np.array([1, -phi1, -phi2])
    ma = np.array([1])
    AR2_process = ArmaProcess(ar, ma)
    
    results = pd.DataFrame(columns=['phi_1_est', 'phi_2_est'])

    for _ in range(n_simulations):
        simulated_data = AR2_process.generate_sample(nsample=n)
        
        g_1_est = pd.Series(simulated_data).autocorr(lag=1)
        g_2_est = pd.Series(simulated_data).autocorr(lag=2)
        
        phi_1_est, phi_2_est = yule_walker_estimation(g_0, g_1_est, g_2_est)
        
        results = results.append({'phi_1_est': phi_1_est, 'phi_2_est': phi_2_est}, ignore_index=True)
    
    return results

In [21]:
phi1_true = 0.5
phi2_true = 0.2

simulation_results = simulate_ar2(phi1_true, phi2_true)

mean_estimates = simulation_results.mean()
mean_estimates


phi_1_est    0.479447
phi_2_est    0.170337
dtype: float64